In [1]:
import sys
sys.path.append('/kaggle/input/inference-asr/Inference')
debug = False

In [2]:
from whisper_jax import FlaxWhisperForConditionalGeneration
#from transformers import FlaxWhisperForConditionalGeneration
from transformers import WhisperFeatureExtractor
import jax.numpy as jnp
import jax
from functions_infer import *
import pandas as pd
from functools import partial
from transformers import AutoTokenizer
from glob import glob
import os

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
pad_to_multiple_of = 1
max_length_gen = 36
batch_size = 20
dtype = jnp.float16
num_workers = 4

# tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-large-v2", language="bn", task="transcribe")
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/tokenizer-asr/token_banglat5/token_banglat5")
tokenizer.bos_token = tokenizer.bos_token_id = None
feature_extractor = WhisperFeatureExtractor.from_pretrained("/kaggle/input/tokenizer-asr/whisper_feat/whisper_feat")

In [4]:
TEST_DIRECTORY = '/kaggle/input/bengaliai-speech/train_mp3s/' if debug else '/kaggle/input/bengaliai-speech/test_mp3s/'
model_path = "/kaggle/input/model-add-data-all/model_all"
paths = glob(os.path.join(TEST_DIRECTORY,'*.mp3'))

In [8]:
df= pd.DataFrame(
    {
        "id":[p.split(os.sep)[-1].replace('.mp3','') for p in paths],
        "size":[os.stat(f).st_size/1024 for f in paths]
    }
)
df = df.sort_values('size')

In [22]:
dataset = Inference(TEST_DIRECTORY,df)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, \
                          collate_fn=partial(collate_fn_infer,feature_extractor=feature_extractor))

In [ ]:
model, params = FlaxWhisperForConditionalGeneration.from_pretrained(
    model_path, dtype=dtype, _do_init=False,)
model.config.forced_decoder_ids = None
model.config.bos_token_id = None
model.config.suppress_tokens = None
model.config.decoder_start_token_id = None
model.generation_config.decoder_start_token_id = [50258, 50302, 50359, 50363]# '<|startoftranscript|><|bn|><|transcribe|><|notimestamps|>
model.generation_config.forced_decoder_ids = None
"""A list of pairs of integers which indicates a mapping from generation indices to token indices 
that will be forced before sampling. For example, [[0, 123]] means the first generated token 
will always be a token of index 123."""
model.generation_config.suppress_tokens = None
model.generation_config.begin_suppress_tokens = None
model.generation_config.bos_token_id = None

In [ ]:
preds_all = []
for audio in train_loader:
    #audio = jnp.array(audio,dtype=dtype)
    generated_ids = model.generate(audio,params=params,max_length=max_length_gen, num_beams=1, do_sample=False).sequences
    transcriptions = tokenizer.batch_decode(generated_ids.tolist(), skip_special_tokens=True)
    preds_all+=transcriptions

In [ ]:
# from bnunicodenormalizer import Normalizer
# bnorm = Normalizer()
# def normalize(sen):
#     _words = [bnorm(word)['normalized']  for word in sen.split()]
#     return " ".join([word for word in _words if word is not None])

def dari(sentence):
    if len(sentence) == 0:
        return sentence
    if sentence[-1]!="।":
        sentence+="।"
    return sentence

In [ ]:
df['sentence'] = preds_all
df = df.drop(columns=['size'])
# df.sentence= df.sentence.apply(lambda x:normalize(x))
df.sentence= df.sentence.apply(dari)

In [ ]:
df.to_csv("submission.csv", index=False)